In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d
Gamma_indices = np.loadtxt("polytropic eoss/gamma_values.txt")
Gamma = Gamma_indices[:,1]

n_0 = 0.16
n_1 = 1.1*n_0
P_1 = 0.3848 * ((2.2)**Gamma[203])
epsilon_1 = 165.3652 - (  (0.84656/(Gamma[203] - 1)) * (1 - (2.2 ** (Gamma[203] - 1))) )
mu_1 = (epsilon_1 + P_1)/n_1

#cs2_1 = 0.2200979082
cs2_1 = Gamma[203]*P_1/(epsilon_1 + P_1)
cs2_N = 0.31

In [ ]:
print(Gamma[203])
print(P_1)
print(epsilon_1)
print(cs2_1)
print(mu_1)

In [ ]:
def generate_points(N, mu_1, mu_N1=2600):
    """
    Generate random mu_points and cs2_points for piecewise linear c_s^2(mu).
    
    Parameters:
    N (int): Number of segments (e.g., 3, 4, 5, 7).
    mu_1 (float): Starting mu value in MeV.
    mu_N1 (float): Ending mu value in MeV.
    
    Returns:
    mu_points (list): List of mu values in increasing order.
    cs2_points (list): List of c_s^2 values corresponding to mu_points.
    """
   
    c_s_max2 = np.random.uniform(0,1)
    print(c_s_max2)
    
    print(c_s_max2)
    if N > 1:
        intermediate_mus = np.random.uniform(mu_1, mu_N1, N-1)
        intermediate_mus.sort()
    else:
        intermediate_mus = []
    mu_points = [mu_1] + list(intermediate_mus) + [mu_N1]

    

    # intermediate_cs2 = np.random.uniform(cs2_1,c_s_max2,N)
    
    
    # cs2_points = [cs2_1] + list(intermediate_cs2)
    
    if N > 1:
         intermediate_cs2 = np.random.uniform(0, c_s_max2, N-1)
    #intermediate_cs2.sort()    
    else:
         intermediate_cs2 = []
    cs2_points = [cs2_1] + list(intermediate_cs2) +[cs2_N]
    
    return mu_points, cs2_points

In [ ]:
# Generate piecewise linear c_s²(μ) parameters
N = 7  # Number of segments, can be 3, 4, 5, or 7
mu_points, cs2_points = generate_points(N, mu_1, mu_N1=2600)


In [ ]:
print(cs2_points)
print(mu_points)

In [ ]:
# #Define c_s²(μ) with linear interpolation
# def c_s2(mu):
    
#     for i in range(len(mu_points)):
#         if mu_points[i] <= mu <= mu_points[i + 1]:
#             return (cs2_points[i] * (mu_points[i + 1] - mu) + cs2_points[i + 1] * (mu - mu_points[i])) / (mu_points[i + 1] - mu_points[i])
    

In [ ]:
def c_s2(mu):
    """Piecewise linear interpolation of c_s^2(μ)"""
    return np.interp(mu, mu_points, cs2_points)

In [ ]:
print(c_s2(1600))

In [ ]:
# Simpson's 1/3 rule
def simpson_integrate(f, a, b, n=200):
    if n % 2 != 0:
        n += 1  # make even
    h = (b - a) / n
    x = np.linspace(a, b, n + 1)
    y = np.array([f(xi) for xi in x])
    return (h / 3) * (y[0] + 4 * np.sum(y[1:-1:2]) + 2 * np.sum(y[2:-2:2]) + y[-1])


In [ ]:
# Eq. (2) integrand
def integrand_eq2(mu_prime):
    cs2 = c_s2(mu_prime)
    if cs2 <= 1e-8:
        raise ValueError("c_s^2 too small or zero at μ = {}".format(mu_prime))
    return 1.0 / (mu_prime * cs2)

# n(μ)
def n_mu(mu):
    if mu <= mu_1:
        return n_1
    integral = simpson_integrate(integrand_eq2, mu_1, mu)
    return n_1 * np.exp(integral)


In [ ]:
# Eq. (3) integrand
def integrand_eq3(mu_prime):
    return n_mu(mu_prime)

# P(μ)
def P_mu(mu):
    if mu <= mu_1:
        return P_1
    integral = simpson_integrate(integrand_eq3, mu_1, mu)
    return P_1 + integral

In [ ]:
# ε1 from thermodynamic relation (μ1 * n1 - P1)
#epsilon_1 = mu_1 * n_1 - P_1

# Eq. (4) integrand
def integrand_eq4(mu_prime):
    cs2 = c_s2(mu_prime)
    if cs2 <= 1e-8:
        raise ValueError(f"c_s^2 too small or zero at μ = {mu_prime}")
    return n_mu(mu_prime) / cs2

# ε(μ)
def epsilon_mu(mu):
    if mu < mu_1:
        raise ValueError("mu must be >= mu_1")
    if abs(mu - mu_1) < 1e-12:  # handle floating point equality
        return epsilon_1
    integral = simpson_integrate(integrand_eq4, mu_1, mu)
    return epsilon_1 + integral

# Example usage:
#mu_test = 1.3  # GeV
#print(f"ε({mu_test} GeV) = {epsilon_mu(mu_test):.6f} GeV/fm^3")


In [ ]:

mu_grid = np.linspace(mu_1, 2600, 200)  # Higher resolution

In [ ]:
print(mu_grid)

In [ ]:
# Compute grids
cs2_grid = np.array([c_s2(mu) for mu in mu_grid])
n_grid = np.array([n_mu(mu) for mu in mu_grid])
P_grid = np.array([P_mu(mu) for mu in mu_grid])

epsilon_grid = np.array([epsilon_mu(mu) for mu in mu_grid])


In [ ]:
print(n_grid)

In [ ]:
print(P_grid)

In [ ]:
print(P_mu(2600))
print(c_s2(2600))
print(epsilon_mu(2600))
print(n_mu(2600)/n_0)
print(P_grid[-1])

In [ ]:
# Convert n to n/n_0
n_over_n0 = n_grid / n_0
print(n_over_n0)

In [ ]:
print(mu_grid)

In [ ]:
print(P_grid * 1.6022e33)

In [ ]:
print(cs2_grid)

In [ ]:
print(epsilon_grid*(1.7827e12))

In [ ]:
# Filter for the desired range: 1.1 n_0 to 40 n_0
mask = (n_over_n0 > 1.1) & (n_over_n0 <= 40)
n_plot = n_over_n0[mask]
P_plot = P_grid[mask]
cs2_plot = cs2_grid[mask]
epsilon_plot = epsilon_grid[mask]
mu_plot = mu_grid[mask]

In [ ]:
print(P_plot[-1])
print(epsilon_plot[-1])
print(n_plot[-1])
print(mu_plot[-1])
print(cs2_plot[-1])

In [ ]:
# Parameters for pQCD
c1 = 0.9008
d1 = 0.5034
d2 = 1.452
nu1 = 0.3553
nu2 = 0.9101

def p_QCD(mu_GeV, X):
    """
    mu_GeV: mu in GeV
    X: scale parameter
    Returns pressure in GeV^4
    """
    prefactor = mu_GeV**4 / (108 * np.pi**2)
    bracket = c1 - (d1 * X**(-nu1)) / (mu_GeV/1.0 - d2 * X**(-nu2))
    return prefactor * bracket

# X values to scan
X_values = np.linspace(1, 4, 100)

mu_target_GeV = 2.6

# pQCD pressures over X
p_values = [p_QCD(mu_target_GeV, X) for X in X_values]

p_min,p_max = min(p_values), max(p_values)

P_at_2600 = P_mu(2600)

is_valid = (p_min *(1.3014e5)<=P_at_2600<=p_max * (1.3014e5))

if is_valid:
    print("EOS is consistent with pQCD at mu = 2600 MeV")
else:
    print("Not consistent with pQCD")





print(p_min * 1.3014e5)
print(p_max * 1.3014e5)
print(P_at_2600)
print(X_values)
print(p_values)

In [ ]:
print(epsilon_plot*(1.7827e12))

In [ ]:
print(cs2_plot)

In [ ]:
print(np.max(cs2_plot))

In [ ]:
p = P_plot * 1.6022e33
epsilon = epsilon_plot * 1.7827e12

In [ ]:
result_array = np.column_stack((p, epsilon, cs2_plot))

In [ ]:
data_BPS = np.loadtxt("data_k")
pres_BPS = data_BPS[:,0]
rho_BPS = data_BPS[:,1]

In [ ]:
data_polytropic_grok = np.loadtxt("polytropic eoss/polytropic_EOS_203.txt")
pres_polytropic = data_polytropic_grok[:,1]
rho_polytropic = data_polytropic_grok[:,0]

In [ ]:
#print(data_polytropic_grok)

In [ ]:
from TOVsolver.unit import g_cm_3, dyn_cm_2, km, Msun, MeV, fm

In [ ]:
pres_total = np.hstack((pres_BPS,pres_polytropic,result_array[:,0]))* dyn_cm_2
eps_total = np.hstack((rho_BPS,rho_polytropic,result_array[:,1]))*g_cm_3
#Last_attempt = np.column_stack((pres_total,eps_total))
#np.savetxt("data_new_1", Last_attempt,  fmt="%.6e")

In [ ]:
print(pres_total)
print(eps_total)
import numpy as np

# Load data
#data_new = np.loadtxt("data_new_1")
#P1 = data_new[:,0] * dyn_cm_2
#rho1 = data_new[:,1] * g_cm_3

P1 = pres_total
rho1 = eps_total

# Check for duplicates in rho1
if len(rho1) != len(np.unique(rho1)):
    print("Duplicate values found in rho1 (density):")
    unique, counts = np.unique(rho1, return_counts=True)
    duplicates = unique[counts > 1]
    print(f"Duplicate density values: {duplicates}")
else:
    print("No duplicates in rho1")

# Check for duplicates in P1
if len(P1) != len(np.unique(P1)):
    print("Duplicate values found in P1 (pressure):")
    unique, counts = np.unique(P1, return_counts=True)
    duplicates = unique[counts > 1]
    print(f"Duplicate pressure values: {duplicates}")
else:
    print("No duplicates in P1")

In [ ]:
# Remove duplicates while keeping the first occurrence
unique_indices = np.unique(rho1, return_index=True)[1]
unique_indices = np.sort(unique_indices)  # Ensure order is preserved
rho1_new = rho1[unique_indices]
P1_new = P1[unique_indices]

print(f"After removing duplicates, rho1 shape: {rho1.shape}, P1 shape: {P1.shape}")

In [ ]:
import EOSgenerators.crust_EOS as crust
import EOSgenerators.RMF_EOS as RMF
import EOSgenerators.Polytrope_EOS as Polytrope
import EOSgenerators.Strangeon_EOS as Strangeon
import TOVsolver.main as main
import matplotlib.pyplot as plt
import numpy as np
import math
from TOVsolver.unit import g_cm_3, dyn_cm_2, km, Msun, MeV, fm

In [ ]:
MR_EOS = main.OutputMR("",rho1_new,P1_new).T
MRT_EOS = main.OutputMRT("",rho1_new,P1_new).T

In [ ]:
plt.plot(MR_EOS[1]/km,MR_EOS[0]/Msun)
plt.show()
print(np.max(MR_EOS[0]/Msun))

In [ ]:
MR_EOS[0]/Msun

In [ ]:
# Plotting
plt.plot( rho1_new/(g_cm_3*(1.7827e12)), P1_new/(dyn_cm_2*(1.6022e33)))
plt.xlabel('Density (MeV/fm³)')
plt.ylabel('Pressure (MeV/fm³)')
plt.title('Pressure vs Density')
plt.xscale("log")
plt.yscale("log")
plt.xlim([10**2,10**4])
plt.ylim([10**0,10**4])
plt.grid(True)
#plt.legend()
plt.show()

In [ ]:
plt.plot(MRT_EOS[1]/Msun, MRT_EOS[2] ,'g-', lw=2,label ="MS1")
plt.ylim([0,3000])
plt.xlim([0.8,3])
plt.show()

In [ ]:
print(rho1/g_cm_3)
print(len(rho1/g_cm_3))
print(len(P1/dyn_cm_2))
print(len(cs2_plot))

In [ ]:
# column_1 = np.ones(1278)  # Placeholder for rho1
# column_2 = np.ones(1278)  # Placeholder for P1
# column_3 = np.ones(194)  # Placeholder for cs2_plot

column_1 = np.ones(1283)  # Placeholder for rho1
column_2 = np.ones(1283)  # Placeholder for P1
column_3 = np.ones(199)  # Placeholder for cs2_plot


# Verify lengths
print("Length of rho1/g_cm_3:", len(column_1))
print("Length of P1/dyn_cm_2:", len(column_2))
print("Length of cs2_plot:", len(column_3))


# Pad cs2_plot to length 1185 with np.nan for the first 1085 elements
cs2_padded = np.full(1283, np.nan)  # Create array of length 1185 filled with NaN
cs2_padded[-199:] = cs2_plot  # Assign last 100 elements from cs2_plot


# Create EOS1 by stacking the arrays
EOS1 = np.column_stack((rho1/g_cm_3, P1/dyn_cm_2, cs2_padded))



# Save EOS1 to a file
np.savetxt("SPEED OF SOUND EOS/EOS_203.txt", EOS1, header="Density(g/cm^3) Pressure(dyn/cm^2) cs2", fmt="%.6e")


#print("EOS3 saved to EOS1_data.txt")

In [ ]:
EOS_194 = np.loadtxt("SPEED OF SOUND EOS/EOS_203.txt")
plt.plot(EOS_194[:,0]/(1.7827e12), EOS_194[:,2])
plt.xscale("log")
plt.xlim([1e2,2e4])
#plt.ylim([0,1])
plt.show()

In [ ]:
import numpy as np

# pQCD parameters
c1 = 0.9008
d1 = 0.5034
d2 = 1.452
nu1 = 0.3553
nu2 = 0.9101

def p_QCD(mu_GeV, X):
    """
    mu_GeV: chemical potential in GeV
    X: scale parameter
    Returns pressure in GeV^4
    """
    prefactor = mu_GeV**4 / (108 * np.pi**2)
    bracket = c1 - (d1 * X**(-nu1)) / (mu_GeV - d2 * X**(-nu2))
    return prefactor * bracket

# Set target mu and X range
mu_target_GeV = 2.6
X_values = np.linspace(1, 4, 100)

# Compute pQCD pressure range in GeV^4
p_values = np.array([p_QCD(mu_target_GeV, X) for X in X_values])
p_min = np.min(p_values)  # in GeV^4
p_max = np.max(p_values)  # in GeV^4

# Convert pQCD pressure range to dyn/cm^2
conversion_factor = 1.3014e5
p_min_dyncm2 = p_min * conversion_factor
p_max_dyncm2 = p_max * conversion_factor

# Verify each EOS
for i in range(1, 181):  # Loop from 1 to 180
    try:
        EOS_data = np.loadtxt(f"SPEED OF SOUND EOS/EOS_{i}.txt")
        P_at_2600 = EOS_data[:, 1][-1] /(1.6022e33) # Pressure in dyn/cm^2
        if p_min_dyncm2 <= P_at_2600 <= p_max_dyncm2:
            print(f"EOS_{i} is consistent with pQCD at mu = 2600 MeV")
        else:
            print(f"NaN")
    except FileNotFoundError:
        continue  # Skip missing files silently

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

web_data_boundary = pd.read_csv("data boundary 1.csv")
web_data_whole = pd.read_csv("data whole.csv")
EOS = np.loadtxt("SPEED OF SOUND EOS/EOS_203.txt")
# Adjust column names based on your CSV structure
energy_density_boundary = web_data_boundary.iloc[:, 0].to_numpy()  # First column: energy density (MeV/fm³)
Pressure_boundary = web_data_boundary.iloc[:, 1].to_numpy()        # Second column: pressure (MeV/fm³)


energy_density_whole = web_data_whole.iloc[:, 0].to_numpy()  # First column: energy density (MeV/fm³)
Pressure_whole = web_data_whole.iloc[:, 1].to_numpy()      


energy_EOS = EOS[:,0]/(1.7827e12)
Pressure_EOS = EOS[:,1]/(1.6022e33)

# Plotting
plt.plot(energy_density_boundary, Pressure_boundary, color = "grey", label = "constrained")
plt.plot(energy_density_whole,Pressure_whole, color = "red",label = "web plot")
plt.plot(energy_EOS, Pressure_EOS, color = "black", label = "whole data")
plt.xlabel('Density (MeV/fm³)')
plt.ylabel('Pressure (MeV/fm³)')
plt.title('Pressure vs Density')
plt.xscale("log")
plt.yscale("log")
plt.xlim([10**2,4e4])
plt.ylim([10**0,10**4])
plt.grid(True)
plt.legend()
plt.show()

In [ ]:
import numpy as np

# pQCD parameters
c1 = 0.9008
d1 = 0.5034
d2 = 1.452
nu1 = 0.3553
nu2 = 0.9101

def p_QCD(mu_GeV, X):
    """
    mu_GeV: chemical potential in GeV
    X: scale parameter
    Returns pressure in GeV^4
    """
    prefactor = mu_GeV**4 / (108 * np.pi**2)
    bracket = c1 - (d1 * X**(-nu1)) / (mu_GeV - d2 * X**(-nu2))
    return prefactor * bracket

# Set target mu and X range
mu_target_GeV = 2.6
X_values = np.linspace(1, 4, 100)

# Compute pQCD pressure range in GeV^4
p_values = np.array([p_QCD(mu_target_GeV, X) for X in X_values])
p_min = np.min(p_values)  # in GeV^4
p_max = np.max(p_values)  # in GeV^4

# Convert pQCD pressure range to dyn/cm^2
conversion_factor = 1.3014e5
p_min_dyncm2 = p_min * conversion_factor
p_max_dyncm2 = p_max * conversion_factor

# Initialize lists to store consistent EOS data
consistent_eos_data = []
consistent_eos_labels = []

# Verify each EOS
for i in range(1, 181):  # Loop from 1 to 180
    try:
        EOS_data = np.loadtxt(f"SPEED OF SOUND EOS/EOS_{i}.txt")
        P_at_2600 = EOS_data[:, 1][-1] /(1.6022e33) # Pressure in dyn/cm^2
        if p_min_dyncm2 <= P_at_2600 <= p_max_dyncm2:
            #print(f"EOS_{i} is consistent with pQCD at mu = 2600 MeV")
            energy_density_2 = EOS_data[:, 0]/ 1.7827e12
            pressure_2 = EOS_data[:, 1] / 1.6022e33
            consistent_eos_data.append((energy_density_2, pressure_2))
            consistent_eos_labels.append(f"EOS_{i}")
            
        #else:
            #print(f"NaN")
    except FileNotFoundError:
        continue  # Skip missing files silently

# Plot consistent EOS
plt.figure()

plt.plot(energy_density_boundary, Pressure_boundary, color = "grey", label = "constrained")
plt.plot(energy_density_whole,Pressure_whole, color = "red", label="web plot" )
# for ed, p, label in zip(consistent_eos_data, consistent_eos_data,consistent_eos_labels):
#     plt.plot(ed[0], p[1],color ="orange")  # Plot energy density vs pressure

# for ed, p in consistent_eos_data:
#     plt.plot(ed, p, color="orange", label="pQCD-consistent EOSs" if ed is consistent_eos_data[0][0] else None)

for i, (ed, p) in enumerate(consistent_eos_data):
    plt.plot(ed, p, color="orange", label="pQCD-consistent EOSs" if i == 0 else None)
    
plt.xlabel('Density (MeV/fm³)')
plt.ylabel('Pressure (MeV/fm³)')
plt.title('Pressure vs Density for pQCD-Consistent EOS at mu = 2600 MeV')
plt.xscale("log")
plt.yscale("log")
plt.xlim([1e2, 4e4])
plt.ylim([1e0, 1e4])
plt.grid(True)
plt.legend()
plt.savefig("EOS PLOT.jpg",dpi=800)
plt.show()

In [ ]:
# import numpy as np
# import pandas as pd
# import matplotlib.pyplot as plt

# # pQCD parameters
# c1 = 0.9008
# d1 = 0.5034
# d2 = 1.452
# nu1 = 0.3553
# nu2 = 0.9101

# def p_QCD(mu_GeV, X):
#     """
#     mu_GeV: chemical potential in GeV
#     X: scale parameter
#     Returns pressure in GeV^4
#     """
#     prefactor = mu_GeV**4 / (108 * np.pi**2)
#     bracket = c1 - (d1 * X**(-nu1)) / (mu_GeV - d2 * X**(-nu2))
#     return prefactor * bracket

# # Set target mu and X range
# mu_target_GeV = 2.6
# X_values = np.linspace(1, 4, 100)

# # Compute pQCD pressure range in GeV^4
# p_values = np.array([p_QCD(mu_target_GeV, X) for X in X_values])
# p_min = np.min(p_values)  # in GeV^4
# p_max = np.max(p_values)  # in GeV^4

# # Convert pQCD pressure range to MeV/fm^3 (1 GeV^4 = 10^12 MeV/fm^3)
# p_min_MeV_fm3 = p_min * 1.3014e5
# p_max_MeV_fm3 = p_max * 1.3014e5

# # Initialize lists to store consistent EOS data
# consistent_eos_data = []
# consistent_eos_labels = []

# # Verify each EOS and collect consistent ones
# for i in range(1, 181):  # Loop from 1 to 180
#     try:
#         EOS_data = np.loadtxt(f"SPEED OF SOUND EOS/EOS_{i}.txt")
#         P_at_2600 = EOS_data[:, 1][-1] / 1.6022e33  # Convert pressure to MeV/fm^3
#         if p_min_MeV_fm3 <= P_at_2600 <= p_max_MeV_fm3:
#             print(f"EOS_{i} is consistent with pQCD at mu = 2600 MeV")
#             # Convert energy density (gm/cm^3 to MeV/fm^3) and pressure (dyn/cm^2 to MeV/fm^3)
#             energy_density = EOS_data[:, 0] / 1.7827e12
#             pressure = EOS_data[:, 1] / 1.6022e33
#             consistent_eos_data.append((energy_density, pressure))
#             consistent_eos_labels.append(f"EOS_{i}")
#         else:
#             print(f"NaN")
#     except FileNotFoundError:
#         continue  # Skip missing files (e.g., EOS_4.txt, EOS_77.txt)

# # Load web data
# try:
#     web_data = pd.read_csv("Web data.csv")
#     energy_density_web = web_data.iloc[:, 0].to_numpy()  # Energy density in MeV/fm^3
#     pressure_web = web_data.iloc[:, 1].to_numpy()        # Pressure in MeV/fm^3
# except FileNotFoundError:
#     print("Web data.csv not found in the current directory")
#     energy_density_web = np.array([])
#     pressure_web = np.array([])

# # Plot all data on a single figure
# plt.figure(figsize=(10, 6))
# # Plot consistent EOS
# for ed, p, label in zip(consistent_eos_data, consistent_eos_data, consistent_eos_labels):
#     plt.plot(ed[0], p[1],color = "red", alpha=0.7)  # Plot EOS data with slight transparency
# # Plot web data
# if len(energy_density_web) > 0:
#     plt.plot(energy_density_web, pressure_web, label="Web Data", color="orange", linewidth=2, linestyle="-")
# plt.xlabel('Density (MeV/fm³)')
# plt.ylabel('Pressure (MeV/fm³)')
# plt.title('Pressure vs Density for pQCD-Consistent EOS and Web Data at mu = 2600 MeV')
# plt.xscale("log")
# plt.yscale("log")
# plt.xlim([1e2, 2e4])
# plt.ylim([1e0, 1e4])
# plt.grid(True)
# plt.legend()
# plt.show()